# 03-sql.ipynb
1. 데이터베이스에서 사용 가능한 테이블과 스키마 가져오기.
1. 질문과 관련된 테이블을 LLM이 결정
1. 해당 테이블의 스키마 확인하기.
1. 질문과 스키마의 정보를 기반으로 쿼리를 생성.
1. LLM을 사용하여 흔히 발생하는 오류가 있는지 SQL 확인.
1. DB에서 SQL을 실행하고 결과를 확인.
1. DB에서 에러 발생시, 수정 후 다시 확인
1. DB 결과를 바탕으로 LLM이 답변 생성

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from langchain_community.utilities import SQLDatabase
import os

DB_URI = os.environ.get('DB_URI')

db = SQLDatabase.from_uri(DB_URI)

print(db.get_usable_table_names())
print(db.run('SELECT * FROM sales LIMIT 5;'))

c:\Users\user\AppData\Local\Programs\Python\Python314\Lib\site-packages\langchain_core\_api\deprecation.py:25: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [19]:
# LLM 초기화
from langchain_openai import ChatOpenAI

model = ChatOpenAI(name='gpt-4.1')

In [20]:
# Agent 용 Tool 만들기
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=model)
toolkit.get_tools()

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000027CC6A8A3C0>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000027CC6A8A3C0>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000027CC6A8A3C0>),
 QuerySQLCheckerTool(description='Use this tool to 

In [21]:
# Agent 만들기
from langchain.agents import create_agent

dialect = db.dialect
top_k = 5

system_prompt = f"""
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer. Unless the user
specifies a specific number of examples they wish to obtain, always limit your
query to at most {top_k} results.

You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while
executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
database.

To start you should ALWAYS look at the tables in the database to see what you
can query. Do NOT skip this step.

Then you should query the schema of the most relevant tables.
"""

agent = create_agent(model, toolkit.get_tools(), system_prompt=system_prompt)

In [22]:
question = '2월에 가장 많이 팔린 물건 3개와, 해당 물건들의 토요일 일요일 평균 매출액'

for event in agent.stream(
    {'messages': [{'role': 'user', 'content': question}]},
    stream_mode='values'
):
    event['messages'][-1].pretty_print()

================================ Human Message =================================

2월에 가장 많이 팔린 물건 3개와, 해당 물건들의 토요일 일요일 평균 매출액
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_v2Zk5rL1S8PGnYn22VQvfcPp)
 Call ID: call_v2Zk5rL1S8PGnYn22VQvfcPp
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

courses, customers, dt_demo, inventory, lotto_draws, members, sales, sample, students, students_courses
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_lJyGxPReGaP0EwvzP3d3XJt3)
 Call ID: call_lJyGxPReGaP0EwvzP3d3XJt3
  Args:
    table_names: sales, inventory
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE inventory (
	id INTEGER GENERATED ALWAYS AS IDENTITY (INCREMENT BY 1 START WITH 1 MINVALUE 1 MAXVALUE 2147483647 CACHE 1 NO CYCLE)